## CS484 Final Project

#### Hardik Paliwal (20725413), Lance Pereira

For our project we have decided to do Project 7, choosing specifically MNIST with a subset of digits. 
 
Our method is .....

Our baseline will be a simple CNN of N+1 (with the K extra classes being labelled as "unknown") to differentiate between unlabeled and labeled classes. Then simply run (some unsupervised model) on the unlabeled data. 

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision as tv
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.autograd import Variable
import sklearn
from torch import optim
import numpy as np
%matplotlib inline

In [2]:
dev=torch.device("cuda") 

In [4]:
trainset = tv.datasets.MNIST(root="./", download=True,train=True,  transform=tv.transforms.Compose(
    [tv.transforms.Resize(32), tv.transforms.ToTensor()]))
trainloader = DataLoader(trainset, batch_size=128, shuffle=True)

testset = tv.datasets.MNIST(root="./", download=True,train=False,  transform=tv.transforms.Compose(
    [tv.transforms.Resize(32), tv.transforms.ToTensor()]))
testloader = DataLoader(testset, batch_size=128, shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [2]:
#Modifies dataset in place to only have values correspounding to the labels in classesToUse
def extractClasses(dataset, classesToUse):
    idx = dataset.targets == classesToUse[0]
    for k in classesToUse:
        idx = (idx | (dataset.targets ==k))
    dataset.targets = dataset.targets[idx]
    dataset.data = dataset.data[idx]
    return dataset

#modifies dataset to change classesToSet labels to -1 (which we will assume stands for unknown class)
def setClassToUnknown(dataset, classesToSet):
  idx = dataset.targets == classesToSet[0]
  for k in classesToSet:
      idx = (idx | (dataset.targets ==k))
  dataset.targets[idx] = -1
  return dataset

In [16]:
#This is an implementation of VGG11 (which is a precursor to VGG16) for mnist dataset.
# it also takes in n, which is the number of classes. N+1 class stands for unknown. 

#this will let us differeniate the unlabelled data from the labelled data, but I am not yet sure how to train on the unlabelled data. More research is needed
class BasicNet(nn.Module):
    def __init__(self, n=9):
        super(BasicNet, self).__init__()
        self.batchNorm = [nn.BatchNorm2d(64), nn.BatchNorm2d(128),nn.BatchNorm2d(256), nn.BatchNorm2d(256),
                          nn.BatchNorm2d(512), nn.BatchNorm2d(512), nn.BatchNorm2d(512), nn.BatchNorm2d(512)]
        self.conv = [
        nn.Conv2d(1, 64, 3, 1, 1) ,nn.Conv2d(64, 128, 3, 1, 1), nn.Conv2d(128, 256, 3, 1, 1), nn.Conv2d(256, 256, 3, 1, 1)
       ,nn.Conv2d(256, 512, 3, 1, 1), nn.Conv2d(512, 512, 3, 1, 1), nn.Conv2d(512, 512, 3, 1, 1), nn.Conv2d(512, 512, 3, 1, 1)
        ]
        maxPool = nn.MaxPool2d(2, stride=2)
        self.conv1 = nn.Sequential(self.conv[0], self.batchNorm[0], nn.ReLU(), maxPool)
        self.conv2 = nn.Sequential(self.conv[1], self.batchNorm[1], nn.ReLU(), maxPool)
        self.conv3 = nn.Sequential(self.conv[2], self.batchNorm[2], nn.ReLU())
        self.conv4 = nn.Sequential(self.conv[3], self.batchNorm[3], nn.ReLU(), maxPool) 
        self.conv5 = nn.Sequential(self.conv[4], self.batchNorm[4], nn.ReLU())
        self.conv6 = nn.Sequential(self.conv[5], self.batchNorm[5], nn.ReLU(), maxPool)
        self.conv7 = nn.Sequential(self.conv[6], self.batchNorm[6], nn.ReLU()) 
        self.conv8 = nn.Sequential(self.conv[7], self.batchNorm[7], nn.ReLU(), maxPool)
        self.fc1 = nn.Linear(512, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, n+1)
        
    def forward(self, x):
        dropOut = nn.Dropout(p=0.5)

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.conv8(x)
        x = torch.flatten(x, 1)
        
        x = dropOut(F.relu(self.fc1(x)))
        x = dropOut(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

In [20]:
def test(data, net):
    net.eval()
    loss_func = nn.CrossEntropyLoss()

    total_correct = 0
    total_loss = 0
    with torch.no_grad():
        correct = 0
        total = 0
        for i, (images, labels) in enumerate(data):
            images= images.to(dev)
            labels = labels.to(dev)
            test_pred = net(images)

            pred = torch.max(test_pred, 1)[1].data.squeeze()
            total_correct+= (pred == labels).sum().item()
            loss = loss_func( test_pred, labels)
            total_loss+= loss.item()*images.size(0)
        return total_correct/len(data.dataset), total_loss/len(data.dataset)
  
def train(num_epochs, net, trainloader, testloader):
    optimizer = optim.SGD(net.parameters(), lr=0.01)
    loss_func = nn.CrossEntropyLoss()

    accuracy_through_epochs = []
    total_step = len(trainloader)
    val_accuracy, val_loss = test(testloader, net)
    train_accuracy, train_loss = test(trainloader, net)
    accuracy_through_epochs.append([0, val_accuracy, val_loss, train_accuracy, train_loss])
    for epoch in range(num_epochs):
        net.train()
        for i, (images, labels) in enumerate(trainloader):
            images= images.to(dev)
            labels = labels.to(dev)
            optimizer.zero_grad()           
            prediction = net(images)
            loss = loss_func( prediction, labels)
            loss.backward()
            optimizer.step()
            if ((i +1) % 100 == 0):
                print(f"Epoch {epoch+1} / {num_epochs}, Step {i+1}/ {total_step} , Loss {loss.item()}")

    val_accuracy, val_loss = test(testloader, net)
    train_accuracy, train_loss = test(trainloader, net)
    accuracy_through_epochs.append([1, val_accuracy, val_loss, train_accuracy, train_loss])
    return accuracy_through_epochs, net

In [21]:
net = BasicNet()
net.to(dev)
result, trained_net = train(4, net, trainloader, testloader)

Epoch 1 / 4, Step 100/ 469 , Loss 0.16441842913627625
Epoch 1 / 4, Step 200/ 469 , Loss 0.1018950492143631
Epoch 1 / 4, Step 300/ 469 , Loss 0.031442608684301376
Epoch 1 / 4, Step 400/ 469 , Loss 0.03806011378765106
Epoch 2 / 4, Step 100/ 469 , Loss 0.015016158111393452
Epoch 2 / 4, Step 200/ 469 , Loss 0.0216631181538105
Epoch 2 / 4, Step 300/ 469 , Loss 0.007580919191241264
Epoch 2 / 4, Step 400/ 469 , Loss 0.02049844153225422
Epoch 3 / 4, Step 100/ 469 , Loss 0.011986605823040009
Epoch 3 / 4, Step 200/ 469 , Loss 0.05962357670068741
Epoch 3 / 4, Step 300/ 469 , Loss 0.007895847782492638
Epoch 3 / 4, Step 400/ 469 , Loss 0.010778998956084251
Epoch 4 / 4, Step 100/ 469 , Loss 0.003865164238959551
Epoch 4 / 4, Step 200/ 469 , Loss 0.004736399743705988
Epoch 4 / 4, Step 300/ 469 , Loss 0.0056166937574744225
Epoch 4 / 4, Step 400/ 469 , Loss 0.031086282804608345
